In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')

In [3]:
fasta_cdna = r'./test/data/s1_cdna.fasta'
fasta_cds = r'./test/data/s1_cds.fasta'

In [4]:
from Bio import SeqIO

In [6]:
dic_cdna = {record.id:record for record in SeqIO.parse(open(fasta_cdna, 'r'), 'fasta')}
dic_cds = {record.id:record for record in SeqIO.parse(open(fasta_cds, 'r'), 'fasta')}

In [142]:
import re
import numpy as np
import pandas as pd
from itertools import product
from mofiwo.utility import generate_utr_from_cdna_cds, load_rna_fasta_zipfile
from mofiwo.analysis import generate_k_mer, generate_feature_by_kmer_loc

In [8]:
dic_utr = generate_utr_from_cdna_cds(dic_cdna, dic_cds)

In [70]:
dic_seq = dict()
for k,v in dic_utr.items():
    dic_seq.update({k: {'CDS': dic_cds[k].seq, 'UT3': v['utr3'], 'UT5': v['utr5']}})

In [104]:
target_feature = dict()
for target_id, seqs in dic_seq.items():
    row_cnt = dict()
    for seq_idx, seq in seqs.items():
        for num_k in [3,4,5]:
            _temp = pd.Series({f'{seq_idx}_{kmer}':len(re.findall(kmer, str(seq))) for kmer in generate_k_mer(num_k)})
            row_cnt.update(_temp.apply(lambda x: x / _temp.sum()).to_dict())
    target_feature[target_id] = row_cnt

In [111]:
foo = pd.DataFrame(target_feature)
foo[foo.apply(lambda x: len(x.index) == 7, axis=1)]

,ENSMUST00000087033


In [129]:
foo.head(2).apply(lambda x: x.index, axis=1)

CDS_AAA     Index(['ENSMUST00000087033'], dtype='object')
CDS_AAAA    Index(['ENSMUST00000087033'], dtype='object')
dtype: object

In [138]:
foo[[len(x) == 7 and x.startswith('CDS') for x in foo.index]].sum()

ENSMUST00000087033    1.0
dtype: float64

In [97]:
bar = pd.Series(foo)

In [101]:
bar2 = bar.apply(lambda x: x/bar.sum())

In [103]:
bar2.to_dict()

{'a': 0.16666666666666666, 'b': 0.3333333333333333, 'c': 0.5}

In [140]:
ret = generate_feature_by_kmer_loc(dic_seq)

In [143]:
zip_cdna = r'E:/workspace/rnamotif/samples/s1_cdna.zip'
zip_cds = r'E:/workspace/rnamotif/samples/s1_cds.zip'

In [144]:
dic_cdna = load_rna_fasta_zipfile(zip_cdna)
dic_cds = load_rna_fasta_zipfile(zip_cds)

In [ ]:
dic_utr = generate_utr_from_cdna_cds(dic_cdna, dic_cds)

In [146]:
dic_seq = dict()
for k,v in dic_utr.items():
    dic_seq.update({k: {'CDS': dic_cds[k].seq, 'UT3': v['utr3'], 'UT5': v['utr5']}})

In [ ]:
ret = generate_feature_by_kmer_loc(dic_seq)

In [ ]:
len(ret)